### Next two cells are necessary to run code. 

### Read instructions for follwoing two cells for decision on what to do

#### Charting information is in different notebook

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests
from bs4 import BeautifulSoup
import re
from io import StringIO
from datetime import datetime, timedelta

In [2]:
class YahooFinanceHistory:
    timeout = 2
    crumb_link = 'https://finance.yahoo.com/quote/{0}/history?p={0}'
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    quote_link = 'https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1d&events=history&crumb={crumb}'

    def __init__(self, symbol, days_back=7):
        self.symbol = symbol
        self.session = requests.Session()
        self.dt = timedelta(days=days_back)

    def get_crumb(self):
        response = self.session.get(self.crumb_link.format(self.symbol), timeout=self.timeout)
        response.raise_for_status()
        match = re.search(self.crumble_regex, response.text)
        if not match:
            raise ValueError('Could not get crumb from Yahoo Finance')
        else:
            self.crumb = match.group(1)

    def get_quote(self):
        if not hasattr(self, 'crumb') or len(self.session.cookies) == 0:
            self.get_crumb()
        now = datetime.utcnow()
        dateto = int(now.timestamp())
        datefrom = int((now - self.dt).timestamp())
        url = self.quote_link.format(quote=self.symbol, dfrom=datefrom, dto=dateto, crumb=self.crumb)
        response = self.session.get(url)
        response.raise_for_status()
        return pd.read_csv(StringIO(response.text), parse_dates=['Date'])

### For single stock entry, enter stock name then add it to stocks list in the cell after this one.

In [10]:
print('Enter Stock')
x = input()
df = YahooFinanceHistory(f'{x}', days_back=500).get_quote()
df.dropna(inplace = True)
df.to_csv(f'{x}.csv', index = False)

Enter Stock


 DOGE-USD


For multiple stock information retrieval. Will update stocks in list for previous 300 days

In [6]:
stocks = [
         ]
for i in stocks:
    df = YahooFinanceHistory(f'{i}', days_back=500).get_quote()
    df.dropna(inplace = True)
    df.to_csv(f'{i}.csv', index = False)
    print(i)

### Use the cell below for alphabetical order to check csvs of all stocks. The stocks list above is not in alphabetical order

In [6]:
sorted(stocks)

['005930.KS',
 'AAL',
 'AAPL',
 'ADM',
 'AM',
 'AMKBY',
 'AMZN',
 'AUDUSD=X',
 'BAC',
 'BAT-USD',
 'BDVSY',
 'BE',
 'BTC-USD',
 'BTCUSD=X',
 'CAIXY',
 'CCL',
 'COG',
 'COP',
 'CQP',
 'CVE',
 'DGB-USD',
 'DOGE-USD',
 'EOS-USD',
 'ETH-USD',
 'ETHUSD=X',
 'GBPUSD=X',
 'GFI',
 'GNT-USD',
 'ICX-USD',
 'JPY=X',
 'LRC-USD',
 'PFE',
 'SNT-USD',
 'TSLA',
 'USDT-USD',
 'VET-USD',
 'XEM-USD',
 'XLM-USD',
 'XRP-USD',
 'ZRX-USD',
 '^VIX']

In [9]:
pd.read_csv('LRC-USD.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-03-10,0.042584,0.052871,0.042584,0.046171,0.046171,10899362
1,2020-03-11,0.046141,0.048500,0.041874,0.044588,0.044588,4511449
2,2020-03-12,0.044633,0.044822,0.024507,0.025021,0.025021,3244805
3,2020-03-13,0.024827,0.033776,0.020367,0.029791,0.029791,3089282
4,2020-03-14,0.029826,0.032278,0.028811,0.029183,0.029183,1460131
...,...,...,...,...,...,...,...
296,2020-12-31,0.179244,0.179284,0.167706,0.169800,0.169800,19734637
297,2021-01-01,0.169802,0.181573,0.167645,0.175754,0.175754,24484684
298,2021-01-02,0.175789,0.179414,0.169980,0.175201,0.175201,18851722
299,2021-01-03,0.175228,0.274570,0.175108,0.272454,0.272454,73010484


In [7]:
pd.read_csv('TSLA.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-03-10,131.886002,133.600006,121.599998,129.065994,129.065994,77972000
1,2020-03-11,128.039993,130.716003,122.599998,126.846001,126.846001,66612500
2,2020-03-12,116.178001,118.900002,109.250000,112.110001,112.110001,94545500
3,2020-03-13,119.000000,121.514000,100.400002,109.323997,109.323997,113201500
4,2020-03-16,93.900002,98.973999,88.433998,89.014000,89.014000,102447500
...,...,...,...,...,...,...,...
203,2020-12-28,674.510010,681.400024,660.799988,663.690002,663.690002,32278600
204,2020-12-29,661.000000,669.900024,655.000000,665.989990,665.989990,22910800
205,2020-12-30,672.000000,696.599976,668.359985,694.780029,694.780029,42846000
206,2020-12-31,699.989990,718.719971,691.119995,705.669983,705.669983,49570900
